# Project 4 Example
### Alec K. Mattu (UTA)
### INST 414
### 11/22/2021

# Instructions
**(1)** add your favorite song to the dataset as the last row. Manually enter the data entries of your favorite song (song name, artist name, year, and lyrics). 

**(2)**  Add a column  L which contains the number of words for each song, column V which contains the number of unique words for each song and column D containing the ratio V/L  (no scaling  needed).

**(3)** Vectorize songs using tfidf vectorization. So now for each song there is a tfidf vector. Store in a dataframe called tfidf_vecs. 

**(4)** Find the cosine similarity of your favorite song to every song in the dataset, using tfidf_vecs. 

**(5-1)** Now add a column to the data called class. We want to label the song classes based on the similarity to your favorite song. Find the median of all cosine similarities and if the similarity of a song is above the median level, label it 1 and if the similarity is less than median level, label it 0.

**(5-2)** Split the data to train and test set (0.25 of the rows in test set). Run simple logistic regression on this dataset, where features are year, L, V, and D and the output variable is class.

**(6)** Print the metrics to evaluate the performance of this classifier.

**(7)** Now select two of the features (out of 4 features) and repeat step 5 and 6. What do you observe? Do the metrics change if you select another two features? What does this change mean? Please add you answer in no more than 100 words, in a markdown cell.

**(8)** Plot the boundary decision for one of the selections in step 7 (your choice) and label the axes.

# Solution

## Part 0 - Preparation

In [2]:
# Import required libraries
import csv
import pandas as pd
import requests

In [2]:
# Hold songs in a list
songs = []

# Open CSV file with handle
with open("dataset_out.csv") as csv_file:
    # Create a file reader
    file_reader = csv.DictReader(csv_file)
    
    # Loop through CSV rows
    for row in file_reader:
        songs.append(row)

## Part 1 - Favorite Song Information
Get the lyrics from azlyrics.com and then insert them into the CSV from Homework 1.

**OPTIONAL: Use the following approach instead of manually adding the song**

In [3]:
# Placeholder dict
song = {
    "Title": "taro",
    "Artist": "alt-j",
    "Year": 2012,
    "Lyrics": "",
}
page = requests.get("https://www.azlyrics.com/lyrics/altj/taro.html")

# Validate the lyric request
if (page.status_code != 200):
    print("Failed to download song lyrics")
    exit()

# Parse the HTML content from the request
soup = bs(page.content, "html.parser")
lyric_div = soup.find(class_= "col-xs-12 col-lg-8 text-center").find("div", attrs = {'class': None})   
lyrics = lyric_div.text.splitlines()
lyrics = list(filter(None, lyrics))

# Recombine the lyrics into a single string
song["Lyrics"] = str.join("\n", lyrics)
songs.append(song)

In [4]:
# Transform list into a DF
df = pd.DataFrame(songs)

In [5]:
# Drop the unnamed first column
df.drop(df.columns[0], axis = 1, inplace = True, errors = 'ignore')

# Drop normalized columns
df.drop(["normalized_L", "normalized_V", "normalized_D"], axis = 1, inplace = True, errors = 'ignore')

# Drop Z-Score columns
df.drop(["z_score_L", "z_score_V", "z_score_D"], axis = 1, inplace = True, errors = 'ignore')

# Drop L, V, D
df.drop(["L", "V", "D"], axis = 1, inplace = True, errors = 'ignore')

In [6]:
df

,Title,Artist,Year,Lyrics
0,the-battle,blood-sweat-tears,1970,While the king and queen lie sleeping\nAnd the...
1,hey-jude,count-basie,1970,"Hey Jude, don't make it bad\nTake a sad song a..."
2,time,david-bowie,1973,"Time, he's waiting in the wings\nHe speaks of ..."
3,we-can-make-the-world-a-whole-lot-brighter,the-brady-bunch,1972,"Birds flying high,\nIn search of a clear blue ..."
4,day-by-day,carmen-mcrae,1972,Day by day I'm falling more in love with you\n...
...,...,...,...,...
4496,tester,anthrax,2006,"I've changed, by staying the same\nWhat does i..."
4497,killing-me-inside,crossfade,2011,There's a dream that comes to me\nAnd it whisp...
4498,babel,cruel-tie,2015,"I'm stepping down, hurrin' up. Settle down. Do..."
4499,i-know-i-ve-been-changed,aaron-neville,2010,"Oh I, know I've been changed\nAnd I know I've ..."
